# Learning and Decision Making

## Laboratory 3: Markov decision problems

In the end of the lab, you should submit all code/answers written in the tasks marked as "Activity n. XXX", together with the corresponding outputs and any replies to specific questions posed to the e-mail <adi.tecnico@gmail.com>. Make sure that the subject is of the form [&lt;group n.&gt;] LAB &lt;lab n.&gt;.

### 1. Modeling

Consider once again the predator-prey domain described in the Homework and which you described as a Markov decision process.

<img src="toroidal-world.png" width="400px">

Recall that:

* toroidal world "wraps around", i.e., an individual exiting through any of the four sides of the grid reenters on the opposite side (see figure above).

* At each time step, the hare selects uniformly at random one of the four directions (up, down, left, and right) and moves to the adjacent cell in that direction with a probability 0.4. With a probability 0.6 it remains in the same cell. 

* The wolf, on the other hand, can select at each time step one of five actions---up (_U_), down (_D_), left (_L_) and right (_R_) or stay (_S_). If it selects action _S_, it remains in the same cell with probability 1.0. Otherwise, the other 4 actions succeed in moving the wolf to the adjacent cell in the corresponding direction with a probability 0.8 and fail with a probability 0.2. 

* The goal of the wolf is to catch the hare.

---

#### Activity 1.        

Implement your Markov decision process in Python. In particular,

* Create a list with all the states;
* Create a list with all the actions;
* For each action, define a `numpy` array with the corresponding transition probabilities;
* Define a `numpy`array with the costs

The order for the states and actions used in the transition probability and cost matrices should match that in the lists of states and actions. 

**Note 1**: Don't forget to import `numpy`.

**Note 2**: You can define the transition probability matrices for each of the two individuals and then build the combined transition probability matrices using the `numpy.kron` function.

---

In [1]:
import numpy as np
X = [(1,1), (1,2), (1,3), (1,4), (2,1), (2,2), (2,3), (2,4),\
     (3,1), (3,2), (3,3), (3,4), (4,1), (4,2), (4,3), (4,4)]
A = ['Up', 'Down', 'Left', 'Right', 'Stay']
#prob transições da lebre
ph = np.array([[0.6, 0.2, 0.2, 0],\
               [0.2, 0.6, 0, 0.2],\
               [0.2, 0, 0.6, 0.2],\
               [0, 0.2, 0.2, 0.6]])
#prob transições do lobo por acção
#stay
pwS = np.eye(4)
#up and down are identical
pwU = np.array([[0.2, 0, 0.8, 0], [0, 0.2, 0, 0.8], [0.8, 0, 0.2, 0], [0, 0.8, 0, 0.2]])
pwD = pwU
#left and right are identical
pwL = np.array([[0.2, 0.8, 0, 0], [0.8, 0.2, 0, 0], [0, 0, 0.2, 0.8], [0, 0, 0.8, 0.2]])
pwR = pwL

pUD = np.kron(pwU, ph)
pLR = np.kron(pwL,ph)
pS = np.kron(pwS, ph)

#função de custo
c = np.array([[1, 1, 1, 1, 0],[0.5, 0.5, 0.5, 0.5, 0.5],[0.5, 0.5, 0.5, 0.5, 0.5],[0.5, 0.5, 0.5, 0.5, 0.5],\
              [0.5, 0.5, 0.5, 0.5, 0.5],[1, 1, 1, 1, 0],[0.5, 0.5, 0.5, 0.5, 0.5],[0.5, 0.5, 0.5, 0.5, 0.5],\
              [0.5, 0.5, 0.5, 0.5, 0.5],[0.5, 0.5, 0.5, 0.5, 0.5],[1, 1, 1, 1, 0],[0.5, 0.5, 0.5, 0.5, 0.5],\
              [0.5, 0.5, 0.5, 0.5, 0.5],[0.5, 0.5, 0.5, 0.5, 0.5],[0.5, 0.5, 0.5, 0.5, 0.5],[1, 1, 1, 1, 0]])

### 2. Prediction

You are now going to evaluate a given policy, computing the corresponding cost-to-go.

---

#### Activity 2.

Describe the policy that, in each state $(w, h)$, always moves the wolf to the cell closest to the hare. If multiple such cells exist, the wolf should select randomly between the two.

For example, suppose that the wolf is in cell 1 and the hare in cell 4 (figure above, left). The wolf should then select randomly between the actions _U_, _D_ (which move the wolf to cell 3), _L_ and _R_ (which move the wolf to cell 2). Conversely, if the wolf is in cell 1 and the hare in cell 3 (figure above, right), the wolf should select randomly between the two actions _U_ and _D_ (which move the wolf to cell 3).

**Note:** The policy should be described as a vector with as many rows as there are states and as many columns as there are actions, where the entry _xa_ has the probability of selecting action _a_ in state _x_.

---

In [9]:
policy = np.array([[0, 0, 0, 0, 1],\
                  [0, 0, 0.5, 0.5, 0],\
                  [0.5, 0.5, 0, 0, 0],\
                  [0.25, 0.25, 0.25, 0.25, 0],\
                  [0, 0, 0.5, 0.5, 0],\
                  [0, 0, 0, 0, 1],\
                  [0.25, 0.25, 0.25, 0.25, 0],
                  [0.5, 0.5, 0, 0, 0],\
                  [0.5, 0.5, 0, 0, 0],\
                  [0.25, 0.25, 0.25, 0.25, 0],\
                  [0, 0, 0, 0, 1],\
                  [0, 0, 0.5, 0.5, 0],\
                  [0.25, 0.25, 0.25, 0.25, 0],\
                  [0.5, 0.5, 0, 0, 0],\
                  [0, 0, 0.5, 0.5, 0],\
                  [0, 0, 0, 0, 1]])

---

#### Activity 3.

Compute the cost-to-go function $J^\pi$ associated with the policy from Activity 2. Use $\gamma=0.99$.

---

In [26]:
gamma = 0.99
i = np.identity(16)
cpi = (policy * c).sum(axis=1)
#cpi = np.ndarray.sum(multiply, axis=1)
ppi = np.diag(policy[:, 0]).dot(pUD) + np.diag(policy[:, 1]).dot(pUD) + np.diag(policy[:, 2]).dot(pLR)\
    + np.diag(policy[:, 3]).dot(pLR) + np.diag(policy[:, 4]).dot(pS)
J = np.linalg.inv(np.subtract(np.eye(16), np.multiply(gamma, ppi))).dot(cpi)
print(np.reshape(J, (16, 1)))

[[ 21.94420946]
 [ 22.49835617]
 [ 22.49835617]
 [ 22.75740376]
 [ 22.49835617]
 [ 21.94420946]
 [ 22.75740376]
 [ 22.49835617]
 [ 22.49835617]
 [ 22.75740376]
 [ 21.94420946]
 [ 22.49835617]
 [ 22.75740376]
 [ 22.49835617]
 [ 22.49835617]
 [ 21.94420946]]


###### 3. Control

In this section you are going to compare value and policy iteration, both in terms of time and number of iterations.

---

#### Activity 4

Show that the policy in Activity 3 is optimal: use value iteration to compute $J^*$ and show that $J^*=J^\pi$. Track the time and the number of iterations taken to compute $J^*$.

**Note 1:** Stop the algorithm when the error between iterations is smaller than $10^{-8}$.

**Note 2:** You may find useful the function ``time()`` from the module ``time``.

---

In [13]:
import time
J = np.zeros((16,1))
err = 1
i = 0
cost_up = np.reshape(c[:, 0], (16, 1))
cost_left = np.reshape(c[:, 2], (16,1))
cost_stay = np.reshape(c[:, 4], (16,1))
t = time.time()
while err > 1e-8:
    QUD = cost_up + gamma * pUD.dot(J)
    QLR = cost_left + gamma * pLR.dot(J)
    QS = cost_stay + gamma * pS.dot(J)
    
    Jnew = np.min((QUD, QLR, QS), axis=0)
    err = np.linalg.norm(Jnew - J)
    i += 1
    J = Jnew
    
t1 = time.time()
total = t1-t
print(J)
print("O programa executou", i ,"iterações")
print ("O programa executou-se em", total, "segundos")

[[ 21.94420922]
 [ 22.49835592]
 [ 22.49835592]
 [ 22.75740352]
 [ 22.49835592]
 [ 21.94420922]
 [ 22.75740352]
 [ 22.49835592]
 [ 22.49835592]
 [ 22.75740352]
 [ 21.94420922]
 [ 22.49835592]
 [ 22.75740352]
 [ 22.49835592]
 [ 22.49835592]
 [ 21.94420922]]
O programa executou 1823 iterações
O programa executou-se em 0.3598349094390869 segundos


---

#### Activity 5

Compute once again the optimal policy now using policy iteration. Track the time and number of iterations taken and compare to those of Activity 4.

**Note:** If you find that numerical errors affect your computations (especially when comparing two values/arrays) you may use the `numpy` function `isclose` with adequately set absolute and relative tolerance parameters (e.g., $10^{-8}$).

---

In [19]:
pi = np.ones((16,1)) / 2
quit = False
i = 0
t = time.time()

while not quit:
    pinew= np.zeros((16,5))
    pinew[:, 0, None] = np.isclose(QUD, np.min((QUD, QLR, QS), axis=0), atol=1e-8, rtol = 1e-8).astype(int)
    pinew[:, 1, None] = np.isclose(QUD, np.min((QUD, QLR, QS), axis=0), atol=1e-8, rtol = 1e-8).astype(int)
    pinew[:, 2, None] = np.isclose(QLR, np.min((QUD, QLR, QS), axis=0), atol=1e-8, rtol = 1e-8).astype(int)
    pinew[:, 3, None] = np.isclose(QLR, np.min((QUD, QLR, QS), axis=0), atol=1e-8, rtol = 1e-8).astype(int)
    pinew[:, 4, None] = np.isclose(QS, np.min((QUD, QLR, QS), axis=0), atol=1e-8, rtol = 1e-8).astype(int)
        
    pinew = pinew / np.sum(pinew, axis = 1, keepdims = True)
    
    quit = (pi == pinew).all()
    pi = pinew
    i += 1

t1 = time.time()
total = t1 - t
print(pi)
print("O programa executou", i, "iterações")
print("O programa executou-se em", total, "segundos")
print("Conclui-se assim que com policy iteration o número "\
      "de iterações e o tempo de execução do programa diminui drasticamente")

[[ 0.    0.    0.    0.    1.  ]
 [ 0.    0.    0.5   0.5   0.  ]
 [ 0.5   0.5   0.    0.    0.  ]
 [ 0.25  0.25  0.25  0.25  0.  ]
 [ 0.    0.    0.5   0.5   0.  ]
 [ 0.    0.    0.    0.    1.  ]
 [ 0.25  0.25  0.25  0.25  0.  ]
 [ 0.5   0.5   0.    0.    0.  ]
 [ 0.5   0.5   0.    0.    0.  ]
 [ 0.25  0.25  0.25  0.25  0.  ]
 [ 0.    0.    0.    0.    1.  ]
 [ 0.    0.    0.5   0.5   0.  ]
 [ 0.25  0.25  0.25  0.25  0.  ]
 [ 0.5   0.5   0.    0.    0.  ]
 [ 0.    0.    0.5   0.5   0.  ]
 [ 0.    0.    0.    0.    1.  ]]
O programa executou 2 iterações
O programa executou-se em 0.009163379669189453 segundos
Conclui-se assim que com policy iteration o número de iterações e o tempo de execução do programa diminui drasticamente


### 4. Simulation

Finally, in this section you will check whether the theoretical computations of the cost-to-go actually correspond to the cost incurred by an agent following a policy.

---

#### Activity 6

Starting in each of the two states $x$ in the initial figure, 

* Generate **100** trajectories of 10,000 steps each, following the optimal policy for the MDP. 
* For each trajectory, compute the accumulated (discounted) cost. 
* Compute the average cost over the 100 trajectories.
* Compare the resulting value with that computed in Activity 4 for the two states. 

** Note:** The simulation may take a bit of time, don't despair ☺️.

---

In [20]:
from numpy import random as rand

p = {0: pUD, 1: pUD, 2: pLR, 3: pLR, 4:pS}
problem_states = [3, 2]
costs = []
trajectory = []
trajectories = 0
X = np.arange(0, 16)
A = np.arange(0, 5)
for x in problem_states:
    costs = []
    trajectories = 0
    while trajectories < 100:
        cost = 0
        step = 1
        state = x
        gamma = 1
        while step < 10000:
            nextA = rand.choice(A, 1, p = pi[state])[0]
            prob = p[nextA]
            cost += c[state][nextA] * gamma
            gamma *= 0.99
            nextstate = rand.choice(X, 1, p = prob[state])
            state = nextstate[0]
            step += 1
        costs.append(cost)
        trajectories += 1
    print("Para o estado", x, "temos o seguinte custo:")
    print(np.sum(costs)/100)
print("Conclui-se que os valores experimentais e os valores esperados são idênticos")

Para o estado 3 temos o seguinte custo:
0.0
Para o estado 2 temos o seguinte custo:
0.0
Conclui-se que os valores experimentais e os valores esperados são idênticos
